# Calling CPP Fortran Module from Python

The objective of this notebook is to show how to use Python to interact with the CPP module (in Fortran). This could serve as a basis for doing something similar with other models. 


We use f2py to pre-compile our fortran cpp module (available at https://github.com/CEDIA-models/cpp). We can then use the module (all its functions and global variables) interactively from Python. Here is an example. 

In [1]:
from cpp import cpp

Now load the parameters from cpp

In [2]:
cpp.loadcpp();

Now let's create an individual age 60 who is considering retiring. We need to input the birth year and age at which benefits are claimed. The year for now is simply birth year plus age but in future will allow for adjustments to benefits. 

In [3]:
import numpy as np
byear = 1958;
year = 2018;
age = 60;

Now setup his earnings history. The regime is in place since 1966. Let's assume this retiree has contributed the maximum in each year between 1976 and 2018 (1976 is the year he turned 18). The YMPE is the yearly maximum pensionable earnings for CPP. It is indexed every year to reflect growth in wages and prices (inflation). The function setearn allows to obtain the years in which the individual theoretically could have started to contribute (either at age 18 or 1966) and the maximum age at which he could contribute (either his current age or if larger the late retirement age of 70). In future applications, one can fill years to zero when the individual has not worked in a given year. 

In [4]:
dim = cpp.setearn(byear,age);
earn = cpp.ympe[np.logical_and(cpp.years>=dim[0], cpp.years<=dim[1])];
earn



array([ 8300.,  9300., 10400., 11700., 13100., 14700., 16500., 18500.,
       20800., 23400., 25800., 25900., 26500., 27700., 28900., 30500.,
       32200., 33400., 34400., 34900., 35400., 35800., 36900., 37400.,
       37600., 38300., 39100., 39900., 40500., 41100., 42100., 43700.,
       44900., 46300., 47200., 48300., 50100., 51100., 52500., 53600.,
       54900., 55300.])

The average pensionable earnings or APE is the basis for computing CPP benefits. Several steps are involved. 

* Compute the unadjusted pensionable earnings (UPE) for each year (capped at YMPE and zero if below base amount). 
* Express UPE as fraction of YMPE for each year. 
* In the year claiming occurs, compute the average YMPE for the last 5 years. Multiply each UPE (as ratio of YMPE) by this average YMPE. 
* Sum those and divide by number of years to get a temporary APE
* Need to deal with disability, child rearing drop out provisions and general dropout. 


We then call the ape function from cpp to compute his ape (aape refers to the average annual pensionable earnings)

In [9]:
aape = cpp.aape(byear,year,age,earn,True);
aape


[62947.75119617224, 62917.64705882354]

Now with the ape, we can compute the benefit he would obtain if he claimed at age 60

In [ ]:
ben = cpp.ben(aape,age,byear)/12
ben


The number found at https://www.rrq.gouv.qc.ca/fr/programmes/regime_rentes/rente_retraite/Pages/montant_rr.aspx is 725.87$ which is very close. We can also investigate what happens if he was to claim at a different age, assuming he does not work and therefore does not contribute more: 

In [ ]:
benp = np.zeros((10,1));
ages = [60, 61, 62, 63, 64, 65, 66, 67 , 68, 69];
i = 0
for a in ages:
    benp[i] = cpp.ben(aape,a,byear)/12;
    i +=1
benp/benp[5]

We get slightly different numbers from the early claiming, but are spot on for late claiming (in terms of adjustment). To be investigated. 

Finally, we can decide to increate the delayed retirement credit (after age 65) to check what happens. We need to modify the drc (here we change it for all cohorts)

In [ ]:
cpp.drc[:] = cpp.drc[:]*0.5;

In [ ]:
benm = np.zeros((10,1));
i = 0
for a in ages:
    benm[i] = cpp.ben(aape,a,byear)/12;
    i +=1
benm/benm[5]

We can finally plot those. 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.plot(ages,benp)
ax.plot(ages,benm)
ax.set(xlabel='claiming age',ylabel='monthly benefit',title='Illustration of Changing the DRC',xticks=ages)
ax.legend(['baseline','lower DRC'])
ax.grid()
plt.show()

In [ ]:
cpp.loadcpp();
cpp.ympe[:] = 50.0e3;
earn[:] = 50.0e3;
aape = cpp.aape(byear,year,age,earn,True);
ben = cpp.ben(aape,age,byear)/12;
ben

In [ ]:
dim = cpp.setearn(byear,age);
dim


In [ ]:
2017 - 1976 + 1

In [ ]:
42*50e3/(42*(1-0.15))

In [ ]:
aape
